In [15]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

- Sử dụng $x^{(2)}$ để minh họa.

In [16]:
x_2 = inputs[1]  # (x^2)

# Khởi tạo dimensions của 3 ma trận trọng số
dim_in = inputs.shape[1]
dim_out = 2

- Khởi tạo 3 ma trận $W_q$, $W_k$, $W_v$, sử dụng _phân phối chuẩn_ $\mathcal{N}(0, 1)$ (68 - 95 - 99.7)

- Vì đang minh họa nên ta set `requires_grad=False`; nhưng khi trong quá trình train LLM thì sẽ set `requires_grad=True` để cập nhật weights trong quá trình huấn luyện.

In [17]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)

print("W_query:", W_query)
print("W_key:", W_key)
print("W_value:", W_value)

W_query: Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
W_key: Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
W_value: Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [18]:
query_2 = x_2 @ W_query  # (x^2) @ W_q
key_2 = x_2 @ W_key      # (x^2) @ W_k
value_2 = x_2 @ W_value  # (x^2) @ W_v
print("query_2:", query_2)
print("key_2:", key_2)
print("value_2:", value_2)

query_2: tensor([0.4306, 1.4551])
key_2: tensor([0.4433, 1.1419])
value_2: tensor([0.3951, 1.0037])


- Tính tất cả `key` vector & `value` vector của tất cả token.

In [19]:
keys = inputs @ W_key      # X @ W_k
values = inputs @ W_value  # X @ W_v
print("keys:", keys)
print("values:", values)

keys: tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
values: tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


- Tính `attention scores` $\omega_{2i}$

In [20]:
# Thay vì dot product giữa query_2 và key_i, ta tính dot product giữa query_2 và tất cả keys
attn_scores_2 = query_2 @ keys.T  # query_2 @ K^T
print("attn_scores:", attn_scores_2)

attn_scores: tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


- Tính `attention weights`

In [21]:
# Chuẩn hóa attention scores 
dim_keys = keys.shape[-1]   # Lấy embedding dimension của keys

# embedding dimension thường là dim cuối cùng của tensor
attn_weights_2 = torch.softmax(attn_scores_2 / dim_keys**0.5, dim=-1)   ## **0.5 là căn bậc hai

print("attn_weights:", attn_weights_2)

attn_weights: tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


- Tính `context vector`

In [22]:
context_vector_2 = attn_weights_2 @ values  # attn_weights_2 @ V
print("context_vector_2:", context_vector_2)

context_vector_2: tensor([0.3061, 0.8210])
